## Example - difi on LSST SSP

[Assumed Inputs](#Assumed-Inputs)  
[Analyzing Observations](#Analyzing-Observations-(Can-I-Find-It%3F))  
[Analyzing Linkages](#Analyzing-Linkages-(Did-I-Find-It%3F))

In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append("..")

import difi

In [2]:
DATA_DIR = "/data/epyc/projects/lsst_ssp/lsst_dm_ssp/notebooks/linking/"

In [3]:
! ls {DATA_DIR}

LSST_HelioLinC.ipynb			Untitled.ipynb	lsstssp.py
LSST_HelioLinC_JPL_dataset-Copy1.ipynb	VC_3days.obs	obs_per_cluster.csv
LSST_HelioLinC_JPL_dataset.ipynb	__pycache__
LSST_SSP_DIFI_VC_3days.txt		lsstssp-bu.py


### Assumed Inputs

Lets take a look at a sample observations file used by the LSST software. 

In [4]:
observations = pd.read_csv(os.path.join(DATA_DIR, "VC_3days.obs"), 
                           sep=" ", 
                           dtype={
                               "obsId" : str,
                               "obj" : str,
                               "objId" : str,
                           },
                           index_col=False)

In [5]:
observations

,obj,time,RA,DEC,x_obs,y_obs,z_obs,night,obsId,objId,r
0,S1009GjOa,52391.002282,171.368899,-12.504937,-0.808597,-0.599239,-0.000026,0,0,72982,1
1,S1001QUsa,52391.002282,169.318742,-13.021633,-0.808597,-0.599239,-0.000026,0,1,16041,1
2,S10036B8a,52391.002282,170.375067,-12.218251,-0.808597,-0.599239,-0.000026,0,2,30951,1
3,S100aAWQa,52391.002282,171.392970,-14.233830,-0.808597,-0.599239,-0.000026,0,3,85187,1
4,S1001DTsa,52391.002282,171.308411,-14.222651,-0.808597,-0.599239,-0.000026,0,4,14030,1
...,...,...,...,...,...,...,...,...,...,...,...
5671056,FD,52393.426183,279.724212,-24.998612,-0.783728,-0.632415,-0.000004,2,5671056,0,1
5671057,FD,52393.426183,278.798817,-26.866569,-0.783728,-0.632415,-0.000004,2,5671057,0,1
5671058,FD,52393.426183,281.781401,-26.198391,-0.783728,-0.632415,-0.000004,2,5671058,0,1
5671059,FD,52393.426183,278.596489,-24.661582,-0.783728,-0.632415,-0.000004,2,5671059,0,1


Before we continue lets tell `difi` what columns to use. Between the observations and linkageMembers dataframe, `difi` needs to know about just three columns:
- linkage_id: the ID assigned to each linkage
- obs_id : the observation ID from which linkages are made
- truth : the truth for every observation

So lets define a dictionary that tells `difi` what columns to use for this information.

In [6]:
columnMapping = {
    # difi column name : data column name
    "linkage_id" : "clusterId",
    "obs_id" : "obsId",
    "truth" : "obj",
    "epoch_mjd" : "time",
}

In [7]:
linkageMembers_temp = pd.read_csv(os.path.join(DATA_DIR, "LSST_SSP_DIFI_VC_3days.txt"), 
                            sep=" ", 
                            index_col=False)

In [8]:
linkageMembers_temp.head()

,clusterId,obsId,r,drdt
0,-1,[ 2370 2529 2561 ... 5668636 5668774 5...,2.0,0
1,0,[341878 344488 349728 352258 748072 750648],2.0,0
2,1,[362637 362707 365191 365250 450378 453063],2.0,0
3,2,[362797 365353 450566 450653 453222 453314],2.0,0
4,3,[362671 362684 362803 365229 365236 365355 450...,2.0,0


Data wrangling into the `difi` format. 

In [9]:
allClusters = linkageMembers_temp[["clusterId", "r", "drdt"]]
linkageMembers_temp = linkageMembers_temp[["clusterId", "obsId"]]

In [10]:
# Split each linkage into its different observation IDs
linkage_list = linkageMembers_temp[columnMapping["obs_id"]].str.strip("[").str.strip("]").str.split().tolist()

# Build initial DataFrame
linkageMembers = pd.DataFrame(pd.DataFrame(linkage_list, index=linkageMembers_temp["clusterId"].values).stack(), columns=[columnMapping["obs_id"]])

# Reset index 
linkageMembers.reset_index(1, drop=True, inplace=True)

# Make linkage_id its own column
linkageMembers[columnMapping["linkage_id"]] = linkageMembers.index

# Re-arrange column order 
linkageMembers = linkageMembers[[columnMapping["linkage_id"], columnMapping["obs_id"]]]

# Not all linkages have the same number of detections, empty detections needs to be dropped
linkageMembers[columnMapping["obs_id"]].replace("", np.nan, inplace=True)
linkageMembers.dropna(inplace=True)
linkageMembers.reset_index(drop=True, inplace=True)


In [11]:
linkageMembers.head(10)

,clusterId,obsId
0,-1,2370
1,-1,2529
2,-1,2561
3,-1,...
4,-1,5668636
5,-1,5668774
6,-1,5669366
7,0,341878
8,0,344488
9,0,349728


In [12]:
keep = linkageMembers["clusterId"].unique()[linkageMembers["clusterId"].value_counts().values >= 4]

In [13]:
linkageMembers = linkageMembers[linkageMembers["clusterId"].isin(keep)]

### Analyzing Observations (Can I Find It?) 

Determing how a linking algorithm performs involves knowing what it should be able to link. `difi` comes with a function that analyzes findablility with a simple assumption: any truths with at least x many observations should be findable. We leave it to the user to determine more complicated findability metrics for their specific science cases. 

Lets see what should be findable with MOPS with this simple metric.

In [14]:
allTruths, summary = difi.analyzeObservations(observations,
                                              minObs=4,
                                              unknownIDs=[],
                                              falsePositiveIDs=["NS", "FD"],
                                              verbose=True,
                                              columnMapping=columnMapping)

Analyzing observations...
Known truth observations: 480144
Unknown truth observations: 0
False positive observations: 5190917
Percent known truth observations (%): 8.467
Percent unknown truth observations (%): 0.000
Percent false positive observations (%): 91.533
Unique truths: 146534
Unique known truths : 146532
Unique known truths with at least 4 detections: 54581

Total time in seconds: 5.259442567825317



The `analyzeObservations` function returns two dataframes. Let's take a look both of them in a little detail. 

The allTruths dataframe lists each unique truth as a row with columns that account for the number of observations that each unique truth has and also if it is findable (if it has more than `minObs` observations). 

In [15]:
allTruths

,obj,num_obs,findable
0,FD,4165797,0
1,NS,1025120,0
2,S100ccNAa,18,1
3,S100377Va,17,1
4,S100sKX4a,17,1
...,...,...,...
146529,S1002cwza,1,0
146530,S100tyCIa,1,0
146531,S100a88ea,1,0
146532,S100FsVHa,1,0


One can trivially select the objects that should or should not be findable thanks to `pandas`.

In [ ]:
findable_known_truths = allTruths[allTruths["findable"] == 1][columnMapping["truth"]].unique()
not_findable_known_truths = allTruths[(allTruths["findable"] == 0) & (~allTruths[columnMapping["truth"]].isin(["-1", "-2"]))][columnMapping["truth"]].unique()

As stated earlier, the `analyzeObservations` function has a very simple findability criteria. The user can write their own metric and then generate a dataframe in the same style as the one above. Doing so will allow `difi` to calculate summary statistics, however, this is not necessary to proceed with determining if truths were linked.

Before we go to seeing how MOPS performed, lets take a look at the summary dataframe. 

In [ ]:
summary

In [ ]:
summary.columns

The summary dataframe is a very simple single row dataframe with some summary numbers about the given observations. 

A few things to note: any observations with truths that are either unknownIDs or falsePositiveIDs are ignored when counting the number of truths that should be findable. For example, in the function call a few cells earlier the `falsePositiveIDs` kwarg was set to `["-1", "-2"]`.

In [ ]:
allTruths[allTruths["findable"] == 0].head()

However... MOPS can't link single detections and has a stricter findability metric. We can write our own function to calculate findability to get a more accurate measure of what should be findable by MOPS.

In [ ]:
# This is where the user's specific science and domain knowledge comes in to
# help determine what is findable and what is not

def calcNight(mjd, midnight=0.166):
    night = mjd + 0.5 - midnight
    return night.astype(int)

def calcFindableMOPS(observations, 
                     trackletMinObs=2, 
                     trackMinNights=3, 
                     falsePositiveIDs=["NS", "FD"],
                     unknownIDs=[],
                     columMapping=columnMapping):
    # Groupby night, then count number of occurences per night
    night_designation_count = observations[~observations[columnMapping["truth"]].isin(falsePositiveIDs + unknownIDs)].groupby(["nid"])[columnMapping["truth"]].value_counts()
    night_designation_count = pd.DataFrame(night_designation_count)
    night_designation_count.rename(columns={columnMapping["truth"]: "num_obs"}, inplace=True)
    night_designation_count.reset_index(inplace=True)

    # Remove nightly detections that would not be linked into a tracklet
    night_designation_count = night_designation_count[night_designation_count["num_obs"] >= trackletMinObs]

    # Groupby object then count number of nights
    try: 
        designation_night_count = pd.DataFrame(night_designation_count.groupby([columnMapping["truth"]])["nid"].value_counts())
    except:
        # No objects satisfy the requirements, return empty array
        return np.array([])
    designation_night_count.rename(columns={"nid": "num_nights"}, inplace=True)
    designation_night_count.reset_index(inplace=True)

    # Grab objects that meet the night requirement
    tracklet_nights_possible = designation_night_count[columnMapping["truth"]].value_counts()
    return tracklet_nights_possible.index[tracklet_nights_possible >= trackMinNights].values

observations["nid"] = calcNight(observations[columnMapping["epoch_mjd"]])
findable_by_mops = calcFindableMOPS(observations)

These are the objects that should actually be findable by MOPS.

In [ ]:
len(findable_by_mops)

Now lets modify the allTruths dataframe to update findability. 

In [ ]:
allTruths["findable"] = np.zeros(len(allTruths), dtype=int)
allTruths.loc[allTruths[columnMapping["truth"]].isin(findable_by_mops), "findable"] = 1

Lets confirm this worked as intended.

In [ ]:
assert len(findable_by_mops) == len(allTruths[allTruths["findable"] == 1])
assert set(findable_by_mops) == set(allTruths[allTruths["findable"] == 1][columnMapping["truth"]].unique())

### Analyzing Linkages (Did I Find It?)

In [16]:
allLinkages, allTruths, summary = difi.analyzeLinkages(observations, 
                                                       linkageMembers, 
                                                       allLinkages=None, 
                                                       allTruths=allTruths,
                                                       summary=summary,
                                                       minObs=4, 
                                                       contaminationThreshold=0.0, 
                                                       unknownIDs=[],
                                                       falsePositiveIDs=["NS", "FD"],
                                                       verbose=True,
                                                       columnMapping=columnMapping)

Analyzing linkages...
Known truth pure linkages: 7
Known truth partial linkages: 0
Unknown truth pure linkages: 0
Unknown truth partial linkages: 0
False positive pure linkages: 0
False positive partial linkages: 0
Mixed linkages: 1752
Total linkages: 1759
Mixed linkage percentage (%): 99.602
Unique known truths linked: 7
Unique known truths missed: 54574
Completeness (%): 0.013

Total time in seconds: 12.514432668685913


The `analyzeLinkages` function returns three dataframes:
- allLinkages: each linkage is summarized as its own row. 
- allTruths: each truth is summarized as its own row. 
- summary: summary statistics in a single row. 

Lets now take a look at each individually.

In [17]:
obs = linkageMembers[linkageMembers["clusterId"].isin(["7739"])]["obsId"].values
observations[observations["obsId"].isin(obs)]

,obj,time,RA,DEC,x_obs,y_obs,z_obs,night,obsId,objId,r


In [18]:
allLinkages[allLinkages["num_obs"] == 2528]

,clusterId,num_members,num_obs,pure,partial,mixed,contamination,linked_truth


For each linkage defined in the `linkageMembers` format, the number of unique 'truths' is counted ('num_members'), the number of unique observations in each linkage ('num_obs'), whether the linkage is 'pure', 'partial' or 'mixed', the contamination percentage (if the linkage is considered 'partial') and if the linkage is either 'pure' or 'partial' then the linked truth ('linked_truth').  

Here we briefly summarize the different linkage types possible:
- 'pure': all observations in a linkage belong to a unique truth
- 'partial': up to a certain percentage of non-unique thruths are allowed so along as one truth has at least the minimum require number of unique observations
- 'mixed': a linkage containing different observations belonging to different truths, we avoid using the word 'false' for these clusters as they may contain unknown truths depending on the use case. We leave interpretation up to the user. 

In [ ]:
allLinkages[allLinkages["pure"] == 1]["linked_truth"]

The allTruths dataframe shows for each truth if it has been found in either a pure or partial linkage. If found in either it sets the found column to 1. 

Lastly, the summary dataframe contains overall statistics on the number of truths found, the completeness (if calculable) and so on...

In [ ]:
summary

Notice that we passed the allTruths dataframe as a kwarg to the `analyzeLinkages` function, this allows the function to access the 'findable' column and calculate completeness. You do not need to pass the allTruths dataframe nor the summary dataframe to use `analyzeLinkages`. Below is an example. 

In [ ]:

allLinkages, allTruths, summary = difi.analyzeLinkages(observations, 
                                                       linkageMembers, 
                                                       allTruths=allTruths,
                                                       minObs=6, 
                                                       contaminationThreshold=0.33, 
                                                       unknownIDs=[],
                                                       falsePositiveIDs=["NS", "FD"],
                                                       verbose=True,
                                                       columnMapping=columnMapping)